In [7]:
import pandas as pd
import json

### Get weather information

In [8]:
def get_weather_data(file, type):
    json_data = []
    with open(file) as f:
        for line in f:
            json_data.append(json.loads(line))
    pd_json = pd.DataFrame()
    flatten_data =[]
    for item in json_data:    
        df_temp = pd.DataFrame()
        df_temp['TIMESTAMP'] = item.keys()
        df_temp[type] = item.values()
        flatten_data.append(df_temp)
    pd_json = pd.concat(flatten_data, axis=0)
    return pd_json

In [9]:
#get all files in the folder
import os
import functools as ft
files = os.listdir('../Dataset/weather/')
dfs=[]
for file in files:
    dftmp = get_weather_data('../Dataset/weather/'+file,file.split('.')[0])
    dfs.append(dftmp)
df_final = ft.reduce(lambda left, right: pd.merge(left, right, on='TIMESTAMP'), dfs)

In [11]:
df_final['date'] = df_final['TIMESTAMP'].apply(lambda x: x.split('T')[0])
df_final['time'] = df_final['TIMESTAMP'].apply(lambda x: x.split('T')[1].split(':')[0])
df_final.drop('TIMESTAMP', axis=1, inplace=True)
df_final.head(5)

,dewptm,hum,pressurem,tempm,wdird,wspdm,date,time
0,13,64,1012,18,210,7.4,2014-08-01,01
1,15,53,1013,22,180,11.1,2014-08-01,13
2,13.0,73,1012,18.0,210,7.4,2014-08-01,01
3,13.0,57,1013,22.0,180,14.8,2014-08-01,13
4,12.0,47,1013,24.0,170,14.8,2014-08-01,13


In [12]:
df_final['dewptm'] = pd.to_numeric(df_final['dewptm'])
df_final['hum'] = pd.to_numeric(df_final['hum'])
df_final['pressurem'] = pd.to_numeric(df_final['pressurem'])
df_final['tempm'] = pd.to_numeric(df_final['tempm'])
df_final['wdird'] = pd.to_numeric(df_final['wdird'])
df_final['wspdm'] = pd.to_numeric(df_final['wspdm'])
df_final.describe()

,dewptm,hum,pressurem,tempm,wdird,wspdm
count,4356.000000,4356.000000,4355.000000,4356.000000,4317.000000,4335.000000
mean,10.355372,70.082645,1013.839265,15.590450,185.276813,12.746897
std,2.823547,14.732925,7.876392,3.595614,86.012201,6.965679
min,0.000000,16.000000,992.000000,2.000000,0.000000,0.000000
25%,9.000000,60.000000,1009.000000,13.000000,110.000000,7.400000
50%,11.000000,72.000000,1013.000000,15.000000,200.000000,11.100000
75%,12.000000,82.000000,1020.000000,18.000000,250.000000,16.700000
max,19.000000,94.000000,1030.000000,27.000000,360.000000,38.900000


In [13]:
df_final.groupby(['date','time']).mean().reset_index().to_csv('../Dataset/weather/weather.csv', index=False)